## Section2 Part0 Generate Reference Files
#### Ke Liu

In [ ]:
import os
import pandas as pd
import seaborn as sns
import subprocess as sp
from os.path import join

### 1 Generate Clean Reference Genome

In [ ]:
genome_fasta = '/home/qukun/liuke/Reference/UCSC/hg38/hg38.fa'
genome_fai = '/home/qukun/liuke/Reference/UCSC/hg38/hg38.fa.fai'

In [ ]:
## extract sequence information from fasta
def get_seq(chrom, start, end, genome_fasta = genome_fasta):
    _shell2output = '/home/qukun/liuke/miniconda3/envs/eccDNA/bin/samtools faidx {0} {1}:{2}-{3}'.format(genome_fasta, chrom, start, end)
    out = sp.check_output(_shell2output, shell = True)
    out = out.decode('utf-8')
    out = out.split('\n')
    title = out.pop(0)
    seq = ''.join(out).upper()
    return seq

## extract sequence of each chromosome in fasta
def genome_weight(genome_fasta = genome_fasta):
    '''
    Calculate the length & weigth of each protien
    '''
    genome_fai = genome_fasta + '.fai'
    genome_df = pd.read_csv(genome_fai,sep= '\t',header=None, names = ["name","length"], index_col = 0, usecols = range(2))
    #Remove the unknown chr 
    genome_df = genome_df.drop(genome_df.filter(regex='_',axis=0).index)
    genome_df['chrom'] = genome_df.index
    genome_df['weight'] = genome_df['length']/genome_df['length'].sum()
    genome_df['seq'] = genome_df.apply(lambda x: get_seq(x['chrom'],1,x["length"]), axis=1)
    return genome_df
genome_df = genome_weight()

## write sequence of each chromosome in fasta
def write_fasta(genome_df,fasta):
    with open(fasta,'w') as f:
        for i in genome_df.index:
            k=0
            f.write('>{0}\n'.format(i))
            for j in range(int(genome_df.loc[i,'length']/50)):
                f.write(genome_df.loc[i,'seq'][50*k:50*(k+1)])
                k=k+1
                f.write('\n')
            f.write(genome_df.loc[i,'seq'][50*k:])
            f.write('\n')
        f.close()

## write chr 1-22 and X sequence to a new fasta
chr_num=list(range(1,23))+['X']
chr_list=['{0}{1}'.format('chr',x) for x in chr_num]
write_fasta(genome_df.loc[chr_list,:],'/home/qukun/liuke/reference/clean/hg38.clean.fa')

### 1 Generate  RepeatMask Bed Files

In [ ]:
## 1 Generate the ReapeatMask Bed Files
def extract_repeats(repeatmasker, output_path):
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    repeatmasker = pd.read_csv(repeatmasker, sep='\t',header=None,index_col=None,skiprows=2)
    repeat_df = pd.DataFrame(columns=['chr','start','end','repeat','class','strand','ID'])
    repeat_df['chr'] = repeatmasker.apply(lambda x: x[0].split()[4],axis=1)
    repeat_df['start'] = repeatmasker.apply(lambda x: x[0].split()[5],axis=1)
    repeat_df['end'] = repeatmasker.apply(lambda x: x[0].split()[6],axis=1)
    repeat_df['repeat'] = repeatmasker.apply(lambda x: x[0].split()[9],axis=1)
    repeat_df['class'] = repeatmasker.apply(lambda x: x[0].split()[10],axis=1)
    repeat_df['strand'] = repeatmasker.apply(lambda x: x[0].split()[8],axis=1)
    repeat_df['ID'] = repeatmasker.apply(lambda x: x[0].split()[-1],axis=1)
    ##
    strand_dict = {'C':'-','+':'+'}
    repeat_df['strand'] = repeat_df.apply(lambda x: strand_dict[x['strand']],axis=1)
    ##
    region_dict = {}
    for region in repeat_df['class'].unique():
        if '?' in region: 
            region_dict[region] = 'Unclear'
        elif '/' in region:
            region_dict[region] =region.split('/')[0]
        else:
            region_dict[region] = region
    repeat_df['class'] = repeat_df.apply(lambda x: region_dict[x['class']],axis=1)
    ##
    for repeat in repeat_df['class'].unique():
        repeat_df[repeat_df['class']==repeat].to_csv(join(output_path,repeat+'.bed'),sep='\t',header=None,index=None)
    return

In [ ]:
## hg38
extract_repeats('/home/qukun/liuke/reference/ucsc/hg38/hg38.fa.out', '/home/qukun/liuke/reference/ucsc/hg38/repeatmask')
sp.check_call('grep L1 /home/qukun/liuke/reference/ucsc/hg38/repeatmask/LINE.bed > /home/qukun/liuke/reference/ucsc/hg38/repeatmask/L1.bed', shell=True)
sp.check_call('grep L2 /home/qukun/liuke/reference/ucsc/hg38/repeatmask/LINE.bed > /home/qukun/liuke/reference/ucsc/hg38/repeatmask/L2.bed', shell=True)
sp.check_call('grep Alu /home/qukun/liuke/reference/ucsc/hg38/repeatmask/SINE.bed > /home/qukun/liuke/reference/ucsc/hg38/repeatmask/Alu.bed', shell=True)
sp.check_call('mv /home/qukun/liuke/reference/ucsc/hg38/repeatmask/Retroposon.bed /home/qukun/liuke/reference/ucsc/hg38/repeatmask/SVA.bed', shell=True)

### 2 Generate genebody Bed Files
##### using Rscript https://github.com/saketkc/gencode_regions

### 3 Generate cpg Bed Files
##### downloading the cpgIslandExt.txt on https://hgdownload.soe.ucsc.edu/goldenPath/hg38/database/

In [ ]:
## Transfer the txt files into BED files
cpg = pd.read_csv('/home/qukun/liuke/reference/GENCODE/hg38/genebody/cpgIslandExt.txt',sep='\t',index_col=0,header=None)
cpg[4]=cpg[4].apply(lambda x: x.replace(' ','_'))
cpg[[1,2,3,4]].to_csv('/home/qukun/liuke/reference/GENCODE/hg38/genebody/cpg.bed', sep='\t', index=None, header=None)